In [1]:
from model import AE
from data_loader import PetDataLoader
from torch.utils.tensorboard import SummaryWriter
import torch
import torchvision
from tqdm import tqdm
from utils import *
import numpy as np

device=0

import os
from datetime import datetime
import subprocess


model = AE(device)
load_model(model, "Mar24_06-21-38_78feb91", "Mar24_06-23-13")
dataloader = PetDataLoader(device, B=256, shuffle=False)
writer = SummaryWriter()

---- init weights ----
   <class 'model.AE'>
   <class 'model.Encoder'>
ok <class 'torch.nn.modules.conv.Conv2d'>
ok <class 'torch.nn.modules.conv.Conv2d'>
ok <class 'torch.nn.modules.conv.Conv2d'>
ok <class 'torch.nn.modules.conv.Conv2d'>
ok <class 'torch.nn.modules.linear.Linear'>
ok <class 'torch.nn.modules.linear.Linear'>
ok <class 'torch.nn.modules.linear.Linear'>
   <class 'model.Decoder'>
ok <class 'torch.nn.modules.linear.Linear'>
ok <class 'torch.nn.modules.conv.ConvTranspose2d'>
ok <class 'torch.nn.modules.conv.ConvTranspose2d'>
ok <class 'torch.nn.modules.conv.ConvTranspose2d'>
ok <class 'torch.nn.modules.conv.ConvTranspose2d'>
   <class 'torch.nn.modules.container.ModuleList'>
using partial dataset
using partial dataset
using partial dataset
using partial dataset
using partial dataset
using partial dataset
using partial dataset
using partial dataset
using partial dataset
using partial dataset
(2027, 3, 64, 64) (2027, 3, 64, 64) (2027, 5)


In [2]:


for batch in tqdm(dataloader):
#     loss = model.train_(batch)
#     print(loss.item())
#     writer.add_scalar("loss", loss.item(), itr)
#     itr += 1
    break

# recons = model.reconst(batch)
# recons = torch.cat([batch["img"][:32].to(device), recons[:32]])
# recons = torchvision.utils.make_grid(recons)

# recons_f = model.reconst_f(batch)
# recons_f = torch.cat([batch["img"][:32].to(device), recons_f[:32]])
# recons_f = torchvision.utils.make_grid(recons_f)

# sample = model.sample_img(batch)
# sample = torchvision.utils.make_grid(sample)

# video = model.sample_vid(batch)

# itr = 0

# writer.add_image('recons', recons, itr)
# writer.add_image('recons_f', recons_f, itr)
# writer.add_image('sample', sample, itr)
# writer.add_video('video', video, itr)
# # writer.add_scalar("loss", loss.item(), itr)

  0%|          | 0/7 [00:00<?, ?it/s]


In [9]:
recons = model.reconst(batch).detach().cpu().numpy().transpose(0,2,3,1)
# recons = model.reconst_phi(batch, -np.pi/6.0).detach().cpu().numpy().transpose(0,2,3,1)
print(recons.shape)

tensor(13.4154, device='cuda:0', grad_fn=<NormBackward0>)
(256, 64, 64, 3)


In [10]:
aux = batch["aux"]
# L = []
# for i in range(256):
#     t, x, y, w, h = aux[i]
#     l = w if w > h else h
#     L.append(l/64.0)
# min(L), max(L) # 1 ~ 8

In [20]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

vid = []

for i in range(256): # 256
    aux = batch["aux"]
    t, x, y, w, h = aux[i]
    l = w if w > h else h
#     x = 1920 - (x+l)

    target = (recons[i] * 255).astype(np.uint8)
    target = cv2.resize(target, (l, l))
#     plt.imshow(target)
#     plt.show()

    target = cv2.resize(target, (l, l))
    bak_orig = (np.zeros((1440, 1920, 3)) + 51).astype(np.uint8)

    bak_orig[y:y+l, x:x+l] += target
    bak_orig = cv2.resize(bak_orig, (960,720))
#     plt.imshow(bak_orig)
#     plt.show()
    vid.append(bak_orig)
#     vid.append(bak_orig.transpose(2,0,1))

vid = np.array(vid)
# vid = np.array([vid])
vid.shape

(256, 720, 960, 3)

In [21]:
vid = np.concatenate([vid, vid2], axis=2)

In [22]:
import moviepy.editor as mpy
def npy_to_gif(npy, filename):
    clip = mpy.ImageSequenceClip(list(npy), fps=10)
    clip.write_gif(filename)
npy_to_gif(vid, "fish_01.gif")

# writer.add_video('fish_0', vid, 0)

t:   1%|          | 2/257 [00:00<00:14, 17.38it/s, now=None]

MoviePy - Building file fish_01.gif with imageio.


In [9]:

t, x, y, w, h = aux[0]

t, x, y, w, h

(tensor(377), tensor(137), tensor(659), tensor(290), tensor(170))

In [ ]:
vid = []
for i in range(4):
    _img = self.sample_img(feed_dict)
    vid.append(_img)
vid = torch.stack(vid)
return vid